In [1]:
!pip install transformers
!pip install torch
!pip install accelerate
!pip install googletrans
!pip install translate

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 23.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 9.2 MB/s eta 0:00:00
  Created wheel for googletrans: filename=googletrans-3.0.0-py3-none-any.whl size=15717 sha256=812a876c7cd00b8eee414b02a889802cb456a1a71d79d38e999137533777d4f4
  Stored in directory: /root/.cache/pip/wheels/b3/81/ea/8b030407f8ebfc2f857814e086bb22ca2d4fea1a7be63652ab
Successfully built googletrans
  Attempting uninstall: chardet
    Found existing installation: chardet 5.2.0
    Uninstalling cha

# Prueba con AutoModelForSeq2Seq

In [2]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, T5Tokenizer
import torch

model_name = "google/flan-t5-small"
model = AutoModelForSeq2SeqLM.from_pretrained(model_name, device_map="auto", torch_dtype=torch.float16)
#tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer = T5Tokenizer.from_pretrained(model_name)

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [3]:

input_text = 'Generate promotion: Come up with an offer for Coca Cola fans'
input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to("cuda")
n = 100
temp = 0.8
# Cantidad de tokens con mayor probabilidad que se considerarán para cada paso de generación
top = 50
sequences = 3

outputs = model.generate(input_ids, max_length=n)

outputs = model.generate(input_ids, max_length=n, do_sample=True)
outputs = model.generate(input_ids, max_length=n, do_sample=True, temperature=temp)
outputs = model.generate(input_ids, max_length=n, temperature=temp, do_sample=True, num_return_sequences=sequences)
outputs = model.generate(input_ids, max_length=n, temperature=temp, do_sample=True, num_return_sequences=sequences, top_k=top)

print(tokenizer.decode(outputs[0]))

<pad> Get a Coca Cola coupon.</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>


# Prueba con T5ForConditionalGeneration


In [4]:
from transformers import T5ForConditionalGeneration

tokenizer1 = T5Tokenizer.from_pretrained(model_name)
model1 = T5ForConditionalGeneration.from_pretrained(model_name, device_map="auto", torch_dtype=torch.float16)

input_text = 'Generate a special deal for Coca Cola'
input_ids = tokenizer1(input_text, return_tensors="pt").input_ids.to("cuda")
n = 100
temp = 0.8
# Cantidad de tokens con mayor probabilidad que se considerarán para cada paso de generación
top = 50
sequences = 3

outputs = model1.generate(input_ids, max_length=n)
outputs = model1.generate(input_ids, max_length=n, do_sample=True)
outputs = model1.generate(input_ids, max_length=n, do_sample=True, temperature=temp)
outputs = model1.generate(input_ids, max_length=n, temperature=temp, do_sample=True, num_return_sequences=sequences)
outputs = model1.generate(input_ids, max_length=n, temperature=temp, do_sample=True, num_return_sequences=sequences, top_k=top)

print(tokenizer1.decode(outputs[0]))

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


<pad> Coca Cola is a company that produces quality ingredients. It specializes in blending chocolate, cocoa, and yogurt as well. Known for its ability to blend sugar, Coca Cola produces ingredients that blend in with other flavors, and serves as a blend of chocolate, chocolate and milk in a variety of flavors.</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>


# Prueba con el modelo Coca

Utilizamos un pipeline

In [18]:

from transformers import pipeline
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, T5Tokenizer, T5ForConditionalGeneration
import torch
from translate import Translator

def traducir_a_espanol(texto):
    translator = Translator(to_lang="es")
    translation = translator.translate(texto)
    return translation

def Crear_texto(input_text):
  model_name ="LeoR2/flan-t5-small-finetuned-Coca"
  model = AutoModelForSeq2SeqLM.from_pretrained(model_name, device_map="auto", torch_dtype=torch.float16)
  #tokenizer = AutoTokenizer.from_pretrained(model_name)
  tokenizer = T5Tokenizer.from_pretrained(model_name)

  # Crear un pipeline para generación de texto a texto
  pipe = pipeline("text2text-generation", model=model, tokenizer=tokenizer, framework='pt')

  # Generar texto a partir de una entrada
  #input_text = 'Generate promotion: Come up with a budget-friendly offer for a Sprite'
  output_text = pipe(input_text)

  # Obtener el texto generado y traducirlo a español
  generated_text = output_text[0]['generated_text']
  translated_text = traducir_a_espanol(generated_text)
  return translated_text

input_text = 'Generate promotion: Come up with a discount of Coca Cola'
Crear_texto(input_text)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


'Consigue un cupón de Coca Cola con un 10% de descuento.'

In [17]:
from translate import Translator

model_name ="LeoR2/flan-t5-small-finetuned-Coca"
model = AutoModelForSeq2SeqLM.from_pretrained(model_name, device_map="auto", torch_dtype=torch.float16)
#tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer = T5Tokenizer.from_pretrained(model_name)

# Crear un pipeline para generación de texto a texto
pipe = pipeline("text2text-generation", model=model, tokenizer=tokenizer, framework='pt')

# Generar texto a partir de una entrada
#input_text = 'Generate promotion: Come up with a budget-friendly offer for a Sprite'
input_text = 'Generate promotion: Come up with a discount of Coca Cola'
output_text = pipe(input_text)
h = output_text[0]['generated_text']
translator = Translator(to_lang="es")
translation = translator.translate(h)
print(translation)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Consigue un cupón de Coca Cola con un 10% de descuento.


In [7]:
type(output_text[0]['generatedtext'])

str

In [8]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, T5Tokenizer, T5ForConditionalGeneration

import torch

model_name ="LeoR2/flan-t5-small-finetuned-Coca"
# model = AutoModelForSeq2SeqLM.from_pretrained(model_name, device_map="auto", torch_dtype=torch.float16)
model = T5ForConditionalGeneration.from_pretrained(model_name, device_map="auto", torch_dtype=torch.float16)
tokenizer = T5Tokenizer.from_pretrained(model_name)

input_text = 'Generate promotion: Come up with an offer for Coca Cola fans'
input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to("cuda")
n = 70
temp = 0.7
# Cantidad de tokens con mayor probabilidad que se considerarán para cada paso de generación
top = 50
sequences = 3

outputs = model.generate(input_ids, max_length=n)
outputs = model.generate(input_ids, max_length=n, do_sample=True)
outputs = model.generate(input_ids, max_length=n, do_sample=True, temperature=temp)
outputs = model.generate(input_ids, max_length=n, temperature=temp, do_sample=True, num_return_sequences=sequences)
outputs = model.generate(input_ids, max_length=n, temperature=temp, do_sample=True, num_return_sequences=sequences, top_k=top)

print(tokenizer.decode(outputs[0]))

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


<pad> The Coca Cola brand has launched a deal to try to promote its products. The deal is being offered to fans of Coca Cola.</s><pad>
